# 2017 Stack Overflow developer survey results cleansing.

In [1]:
import os
import zipfile
import pandas as pd

ARCHIVE_PATH = 'data/developer_survey_2017.zip'
RESULTS_PATH = 'data/unpacked/survey_results_public.csv'
SCHEMA_PATH = 'data/unpacked/survey_results_schema.csv'
CLEAN_PATH = 'data/unpacked/cleaned_survey_results_public.csv'
FEATURE_COLUMNS = [
    'Professional',
    'ProgramHobby',
    'Country',
    'University',
    'FormalEducation',
    'MajorUndergrad',
    'YearsProgram'
]
LABEL_NAME = 'DeveloperType'

In [2]:
# Unpack Archive if not already unpacked
if os.path.isfile(RESULTS_PATH) is not True:
    zip_ref = zipfile.ZipFile(ARCHIVE_PATH, 'r')
    zip_ref.extractall('data/unpacked')
    zip_ref.close()

In [3]:
# Parse the local CSV file.
if not os.path.isfile(CLEAN_PATH):
    raw_data = pd.read_csv(
        filepath_or_buffer=RESULTS_PATH,
        header=0,
        low_memory=False
    )
    output = FEATURE_COLUMNS
    output.append(LABEL_NAME)
    raw_data.loc[:, output].to_csv(CLEAN_PATH, index=False)
else:
    raw_data = pd.read_csv(
        filepath_or_buffer=CLEAN_PATH,
        header=0,
        low_memory=False
    )

In [4]:
print(raw_data.shape)
display(raw_data)

(51392, 9)


,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,YearsCodedJobPast,DeveloperType
0,Student,"Yes, both",United States,No,Secondary school,NaN,2 to 3 years,NaN,NaN
1,Student,"Yes, both",United Kingdom,"Yes, full-time",Some college/university study without earning ...,Computer science or software engineering,9 to 10 years,NaN,NaN
2,Professional developer,"Yes, both",United Kingdom,No,Bachelor's degree,Computer science or software engineering,20 or more years,NaN,Other
3,Professional non-developer who sometimes write...,"Yes, both",United States,No,Doctoral degree,A non-computer-focused engineering discipline,14 to 15 years,NaN,NaN
4,Professional developer,"Yes, I program as a hobby",Switzerland,No,Master's degree,Computer science or software engineering,20 or more years,NaN,Mobile developer; Graphics programming; Deskto...
5,Student,"Yes, both",New Zealand,"Yes, full-time",Secondary school,NaN,6 to 7 years,NaN,NaN
6,Professional non-developer who sometimes write...,"Yes, both",United States,No,Master's degree,A non-computer-focused engineering discipline,9 to 10 years,NaN,NaN
7,Professional developer,"Yes, both",Poland,No,Master's degree,Computer science or software engineering,10 to 11 years,NaN,Web developer
8,Professional developer,"Yes, I program as a hobby",Colombia,"Yes, part-time",Bachelor's degree,Computer science or software engineering,13 to 14 years,NaN,Web developer; Mobile developer
9,Professional developer,"Yes, I program as a hobby",France,"Yes, full-time",Master's degree,Computer science or software engineering,13 to 14 years,NaN,Mobile developer; Desktop applications developer


In [5]:
# Remove all rows with no label values
raw_data = raw_data.dropna(subset=[LABEL_NAME], how='all')
print(raw_data.shape)
display(raw_data)

(36125, 9)


,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,YearsCodedJobPast,DeveloperType
2,Professional developer,"Yes, both",United Kingdom,No,Bachelor's degree,Computer science or software engineering,20 or more years,NaN,Other
4,Professional developer,"Yes, I program as a hobby",Switzerland,No,Master's degree,Computer science or software engineering,20 or more years,NaN,Mobile developer; Graphics programming; Deskto...
7,Professional developer,"Yes, both",Poland,No,Master's degree,Computer science or software engineering,10 to 11 years,NaN,Web developer
8,Professional developer,"Yes, I program as a hobby",Colombia,"Yes, part-time",Bachelor's degree,Computer science or software engineering,13 to 14 years,NaN,Web developer; Mobile developer
9,Professional developer,"Yes, I program as a hobby",France,"Yes, full-time",Master's degree,Computer science or software engineering,13 to 14 years,NaN,Mobile developer; Desktop applications developer
11,Professional developer,No,Canada,No,Bachelor's degree,Computer science or software engineering,13 to 14 years,NaN,Web developer
13,Professional developer,"Yes, both",Germany,No,Some college/university study without earning ...,Computer science or software engineering,15 to 16 years,NaN,Web developer
14,Professional developer,"Yes, I program as a hobby",United Kingdom,No,Professional degree,Computer engineering or electrical/electronics...,20 or more years,NaN,Embedded applications/devices developer
15,Professional developer,"Yes, I program as a hobby",United States,"Yes, part-time",Primary/elementary school,NaN,11 to 12 years,NaN,Desktop applications developer
16,Professional developer,"Yes, both",United Kingdom,No,Secondary school,NaN,8 to 9 years,NaN,Web developer


In [6]:
# Exapnd rows that have multi labels into new records
# There is a total possiblity of 14 labels per row
# TODO: Revisit this crazyness ^ to understand if expanding 1 => 14 makes sense
expanded_data = []
for (idx, row) in raw_data.iterrows():
    # Check for delimiter
    split = [x.strip() for x in row.loc[LABEL_NAME].split(';')]
    # Where did we come up with the magic number 5? See the comments above.
    if len(split) > 1:
        # expand deliminated values into uinque rows
        for label in split:
            new_row = row.copy()
            new_row[LABEL_NAME]= label
            expanded_data.append(new_row)
    elif len(split) is 1:
        expanded_data.append(row)
        
raw_data = pd.DataFrame(expanded_data).reset_index(drop=True)
print(raw_data.shape)
display(raw_data)

(77259, 9)


,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,YearsCodedJobPast,DeveloperType
0,Professional developer,"Yes, both",United Kingdom,No,Bachelor's degree,Computer science or software engineering,20 or more years,NaN,Other
1,Professional developer,"Yes, I program as a hobby",Switzerland,No,Master's degree,Computer science or software engineering,20 or more years,NaN,Mobile developer
2,Professional developer,"Yes, I program as a hobby",Switzerland,No,Master's degree,Computer science or software engineering,20 or more years,NaN,Graphics programming
3,Professional developer,"Yes, I program as a hobby",Switzerland,No,Master's degree,Computer science or software engineering,20 or more years,NaN,Desktop applications developer
4,Professional developer,"Yes, both",Poland,No,Master's degree,Computer science or software engineering,10 to 11 years,NaN,Web developer
5,Professional developer,"Yes, I program as a hobby",Colombia,"Yes, part-time",Bachelor's degree,Computer science or software engineering,13 to 14 years,NaN,Web developer
6,Professional developer,"Yes, I program as a hobby",Colombia,"Yes, part-time",Bachelor's degree,Computer science or software engineering,13 to 14 years,NaN,Mobile developer
7,Professional developer,"Yes, I program as a hobby",France,"Yes, full-time",Master's degree,Computer science or software engineering,13 to 14 years,NaN,Mobile developer
8,Professional developer,"Yes, I program as a hobby",France,"Yes, full-time",Master's degree,Computer science or software engineering,13 to 14 years,NaN,Desktop applications developer
9,Professional developer,No,Canada,No,Bachelor's degree,Computer science or software engineering,13 to 14 years,NaN,Web developer


In [7]:
# One-Hot Encode
raw_data = pd.get_dummies(raw_data)
print(raw_data.shape)
display(raw_data)

(77259, 258)


,Professional_Professional developer,Professional_Professional non-developer who sometimes writes code,Professional_Student,ProgramHobby_No,"ProgramHobby_Yes, I contribute to open source projects","ProgramHobby_Yes, I program as a hobby","ProgramHobby_Yes, both",Country_Afghanistan,Country_Aland Islands,Country_Albania,...,DeveloperType_Developer with a statistics or mathematics background,DeveloperType_Embedded applications/devices developer,DeveloperType_Graphic designer,DeveloperType_Graphics programming,DeveloperType_Machine learning specialist,DeveloperType_Mobile developer,DeveloperType_Other,DeveloperType_Quality assurance engineer,DeveloperType_Systems administrator,DeveloperType_Web developer
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
